## Evaluation
Submissions are scored on the root mean squared error.

## Blueprint
1. Split the dataset into features(full values) and targets(missing values)

Loop for the each target,<br>

2. Extract top3 features using correlation
3. Seperate rows having missing values for prediction
4. Apply ML models

### Import Packages

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from scipy.special import expit
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression

# Show all the columns and rows
pd.set_option("display.max_columns", None)  # columns
# pd.set_option("display.max_rows", None)   # rows

## 1. Data Loading

In [2]:
# Load dataset
data = pd.read_csv('data.csv')

print(data.shape)
data.head()

(1000000, 81)


,row_id,F_1_0,F_1_1,F_1_2,F_1_3,F_1_4,F_1_5,F_1_6,F_1_7,F_1_8,F_1_9,F_1_10,F_1_11,F_1_12,F_1_13,F_1_14,F_2_0,F_2_1,F_2_2,F_2_3,F_2_4,F_2_5,F_2_6,F_2_7,F_2_8,F_2_9,F_2_10,F_2_11,F_2_12,F_2_13,F_2_14,F_2_15,F_2_16,F_2_17,F_2_18,F_2_19,F_2_20,F_2_21,F_2_22,F_2_23,F_2_24,F_3_0,F_3_1,F_3_2,F_3_3,F_3_4,F_3_5,F_3_6,F_3_7,F_3_8,F_3_9,F_3_10,F_3_11,F_3_12,F_3_13,F_3_14,F_3_15,F_3_16,F_3_17,F_3_18,F_3_19,F_3_20,F_3_21,F_3_22,F_3_23,F_3_24,F_4_0,F_4_1,F_4_2,F_4_3,F_4_4,F_4_5,F_4_6,F_4_7,F_4_8,F_4_9,F_4_10,F_4_11,F_4_12,F_4_13,F_4_14
0,0,-0.354591,-0.464038,2.304115,0.734486,1.696395,0.136285,-0.518344,0.502640,-1.852504,-0.500665,-1.416075,1.201521,0.551902,-0.759827,NaN,2,2,0,3,2,1,1,3,2,2,3,3,2,5,4,1,0,1,0,2,1,2,0,1,2,-0.240522,0.061529,0.561090,-0.171943,-0.431996,0.473508,0.596924,0.819306,1.479061,1.264616,-1.116881,0.759443,-0.086915,-0.620685,0.057216,1.076380,-0.780608,-1.940907,-0.717021,0.599093,0.498347,0.118770,-0.228913,NaN,0.301610,5.547214,1.066871,-0.134313,-0.101040,-0.660871,3.744152,0.794438,0.265185,-0.561809,0.196480,0.373434,6.206995,3.809505,1.236486,1.182055
1,1,1.380940,-0.499626,-0.418548,1.911725,-0.826130,-1.715371,-0.577091,-1.041486,0.596067,-0.363425,-0.853630,0.674525,0.843058,-0.041438,0.259496,3,5,2,1,4,2,3,2,0,0,9,3,1,4,1,2,1,0,1,1,1,1,3,2,5,-0.446068,1.433358,0.040099,-1.994062,-0.602324,-0.611391,-1.151884,0.065485,0.352023,-0.843751,1.167272,0.921445,-0.839827,0.759015,-1.547387,0.720435,-0.944045,1.796462,-1.046357,-0.581515,0.704543,0.375222,0.705963,0.032771,NaN,-1.707374,-1.188114,-0.562419,-1.462988,1.290672,-2.895826,-0.738275,2.361818,-0.060753,0.727249,-0.271882,5.232157,-4.218259,-2.724883,-0.063775
2,2,0.256023,-1.059874,NaN,0.345678,1.513814,1.243864,-0.509648,-0.800481,-0.115945,0.595777,-0.073235,-1.381605,-0.108676,0.703693,-0.464042,3,2,1,3,2,1,0,6,1,1,3,4,1,2,2,1,2,2,4,8,1,5,1,2,4,-0.770592,0.483139,-0.636484,-1.305018,-2.089889,0.276761,-1.208640,-0.855769,0.232363,0.215841,-1.031405,0.582437,-0.314639,-0.497409,0.489356,0.915049,-0.513410,0.904206,-0.056089,0.212927,-0.574126,-1.517749,-0.888472,0.142264,1.000822,1.914908,3.877128,NaN,0.358635,0.443973,2.252834,0.472496,2.491386,0.353381,-0.260682,-0.000833,-0.116457,-2.131747,3.661499,-0.131576
3,3,-0.728420,-2.432399,-2.453602,-0.020509,0.333397,0.086049,-1.787601,0.667011,0.761564,-2.217847,-0.618973,0.742112,0.494157,0.744673,-0.769181,2,3,2,1,1,1,2,0,6,2,6,2,1,1,2,3,1,2,1,2,6,2,1,0,2,-0.691020,-1.003026,0.868989,0.464920,NaN,0.499838,0.203723,-0.451576,-1.233499,0.903493,1.144558,-1.479893,-0.414316,0.024820,-0.976814,0.176633,-0.940022,-1.918049,1.506448,0.604388,0.578250,-0.122676,-0.711811,0.228523,0.999599,-2.638262,0.546676,0.865400,-0.857077,2.667105,2.004600,-4.664806,-0.847211,-0.264249,0.664334,-0.557868,8.499483,-4.738799,-3.054611,0.494152
4,4,0.590212,-0.066127,0.468009,-1.096038,0.119399,-1.809710,0.466358,-0.053196,-0.580320,-1.143500,1.338692,1.193330,1.038010,-0.763692,0.513466,5,0,1,3,3,0,2,5,1,3,5,5,4,4,2,0,2,3,2,5,2,2,1,0,4,1.196402,1.076537,0.486482,0.424305,0.475876,0.426978,-1.668823,-0.843056,1.008980,-0.704152,-0.894834,1.491608,-0.149938,1.256330,2.427308,0.878677,0.594314,0.380175,0.666499,-0.664403,1.276316,NaN,-1.276574,-0.768874,1.852815,-0.230342,-0.459019,1.128705,-0.748683,NaN,0.976937,2.558883,3.377724,0.846891,0.696032,0.554121,-5.979714,-2.869631,3.733057,-0.722943


In [3]:
# # Store row_id
# id = data.row_id

# Drop 'row_id'
data = data.drop(columns=['row_id'])
data.head()

,F_1_0,F_1_1,F_1_2,F_1_3,F_1_4,F_1_5,F_1_6,F_1_7,F_1_8,F_1_9,F_1_10,F_1_11,F_1_12,F_1_13,F_1_14,F_2_0,F_2_1,F_2_2,F_2_3,F_2_4,F_2_5,F_2_6,F_2_7,F_2_8,F_2_9,F_2_10,F_2_11,F_2_12,F_2_13,F_2_14,F_2_15,F_2_16,F_2_17,F_2_18,F_2_19,F_2_20,F_2_21,F_2_22,F_2_23,F_2_24,F_3_0,F_3_1,F_3_2,F_3_3,F_3_4,F_3_5,F_3_6,F_3_7,F_3_8,F_3_9,F_3_10,F_3_11,F_3_12,F_3_13,F_3_14,F_3_15,F_3_16,F_3_17,F_3_18,F_3_19,F_3_20,F_3_21,F_3_22,F_3_23,F_3_24,F_4_0,F_4_1,F_4_2,F_4_3,F_4_4,F_4_5,F_4_6,F_4_7,F_4_8,F_4_9,F_4_10,F_4_11,F_4_12,F_4_13,F_4_14
0,-0.354591,-0.464038,2.304115,0.734486,1.696395,0.136285,-0.518344,0.502640,-1.852504,-0.500665,-1.416075,1.201521,0.551902,-0.759827,NaN,2,2,0,3,2,1,1,3,2,2,3,3,2,5,4,1,0,1,0,2,1,2,0,1,2,-0.240522,0.061529,0.561090,-0.171943,-0.431996,0.473508,0.596924,0.819306,1.479061,1.264616,-1.116881,0.759443,-0.086915,-0.620685,0.057216,1.076380,-0.780608,-1.940907,-0.717021,0.599093,0.498347,0.118770,-0.228913,NaN,0.301610,5.547214,1.066871,-0.134313,-0.101040,-0.660871,3.744152,0.794438,0.265185,-0.561809,0.196480,0.373434,6.206995,3.809505,1.236486,1.182055
1,1.380940,-0.499626,-0.418548,1.911725,-0.826130,-1.715371,-0.577091,-1.041486,0.596067,-0.363425,-0.853630,0.674525,0.843058,-0.041438,0.259496,3,5,2,1,4,2,3,2,0,0,9,3,1,4,1,2,1,0,1,1,1,1,3,2,5,-0.446068,1.433358,0.040099,-1.994062,-0.602324,-0.611391,-1.151884,0.065485,0.352023,-0.843751,1.167272,0.921445,-0.839827,0.759015,-1.547387,0.720435,-0.944045,1.796462,-1.046357,-0.581515,0.704543,0.375222,0.705963,0.032771,NaN,-1.707374,-1.188114,-0.562419,-1.462988,1.290672,-2.895826,-0.738275,2.361818,-0.060753,0.727249,-0.271882,5.232157,-4.218259,-2.724883,-0.063775
2,0.256023,-1.059874,NaN,0.345678,1.513814,1.243864,-0.509648,-0.800481,-0.115945,0.595777,-0.073235,-1.381605,-0.108676,0.703693,-0.464042,3,2,1,3,2,1,0,6,1,1,3,4,1,2,2,1,2,2,4,8,1,5,1,2,4,-0.770592,0.483139,-0.636484,-1.305018,-2.089889,0.276761,-1.208640,-0.855769,0.232363,0.215841,-1.031405,0.582437,-0.314639,-0.497409,0.489356,0.915049,-0.513410,0.904206,-0.056089,0.212927,-0.574126,-1.517749,-0.888472,0.142264,1.000822,1.914908,3.877128,NaN,0.358635,0.443973,2.252834,0.472496,2.491386,0.353381,-0.260682,-0.000833,-0.116457,-2.131747,3.661499,-0.131576
3,-0.728420,-2.432399,-2.453602,-0.020509,0.333397,0.086049,-1.787601,0.667011,0.761564,-2.217847,-0.618973,0.742112,0.494157,0.744673,-0.769181,2,3,2,1,1,1,2,0,6,2,6,2,1,1,2,3,1,2,1,2,6,2,1,0,2,-0.691020,-1.003026,0.868989,0.464920,NaN,0.499838,0.203723,-0.451576,-1.233499,0.903493,1.144558,-1.479893,-0.414316,0.024820,-0.976814,0.176633,-0.940022,-1.918049,1.506448,0.604388,0.578250,-0.122676,-0.711811,0.228523,0.999599,-2.638262,0.546676,0.865400,-0.857077,2.667105,2.004600,-4.664806,-0.847211,-0.264249,0.664334,-0.557868,8.499483,-4.738799,-3.054611,0.494152
4,0.590212,-0.066127,0.468009,-1.096038,0.119399,-1.809710,0.466358,-0.053196,-0.580320,-1.143500,1.338692,1.193330,1.038010,-0.763692,0.513466,5,0,1,3,3,0,2,5,1,3,5,5,4,4,2,0,2,3,2,5,2,2,1,0,4,1.196402,1.076537,0.486482,0.424305,0.475876,0.426978,-1.668823,-0.843056,1.008980,-0.704152,-0.894834,1.491608,-0.149938,1.256330,2.427308,0.878677,0.594314,0.380175,0.666499,-0.664403,1.276316,NaN,-1.276574,-0.768874,1.852815,-0.230342,-0.459019,1.128705,-0.748683,NaN,0.976937,2.558883,3.377724,0.846891,0.696032,0.554121,-5.979714,-2.869631,3.733057,-0.722943


## 2. Data Exploration

In [4]:
# Columns with full/missing values
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 80 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   F_1_0   981603 non-null   float64
 1   F_1_1   981784 non-null   float64
 2   F_1_2   981992 non-null   float64
 3   F_1_3   981750 non-null   float64
 4   F_1_4   981678 non-null   float64
 5   F_1_5   981911 non-null   float64
 6   F_1_6   981867 non-null   float64
 7   F_1_7   981872 non-null   float64
 8   F_1_8   981838 non-null   float64
 9   F_1_9   981751 non-null   float64
 10  F_1_10  982039 non-null   float64
 11  F_1_11  981830 non-null   float64
 12  F_1_12  981797 non-null   float64
 13  F_1_13  981602 non-null   float64
 14  F_1_14  981961 non-null   float64
 15  F_2_0   1000000 non-null  int64  
 16  F_2_1   1000000 non-null  int64  
 17  F_2_2   1000000 non-null  int64  
 18  F_2_3   1000000 non-null  int64  
 19  F_2_4   1000000 non-null  int64  
 20  F_2_5   1000000 non-null 

### - Location of missing values

In [5]:
# # Return 'Row'-'Column'
# def where_missing(df):

#     m_list = []

#     for row in data.index.tolist():
#         for col in [c for c in df]:

#             value = df.loc[row, col]

#             if pd.isnull(value):        # if the value is missing
#                 m_list.append(str(row)+'-'+col)
                
#     return m_list 

In [6]:
# loc_missing = where_missing(data)

## 3. Build the algorithm

### 1) Split into features and targets

In [7]:
# Extract the feature dataframe
data_features = data.loc[:, 'F_2_0':'F_2_24']   # dataframe with non-missing values

print(data_features.shape)
data_features.head()

(1000000, 25)


,F_2_0,F_2_1,F_2_2,F_2_3,F_2_4,F_2_5,F_2_6,F_2_7,F_2_8,F_2_9,F_2_10,F_2_11,F_2_12,F_2_13,F_2_14,F_2_15,F_2_16,F_2_17,F_2_18,F_2_19,F_2_20,F_2_21,F_2_22,F_2_23,F_2_24
0,2,2,0,3,2,1,1,3,2,2,3,3,2,5,4,1,0,1,0,2,1,2,0,1,2
1,3,5,2,1,4,2,3,2,0,0,9,3,1,4,1,2,1,0,1,1,1,1,3,2,5
2,3,2,1,3,2,1,0,6,1,1,3,4,1,2,2,1,2,2,4,8,1,5,1,2,4
3,2,3,2,1,1,1,2,0,6,2,6,2,1,1,2,3,1,2,1,2,6,2,1,0,2
4,5,0,1,3,3,0,2,5,1,3,5,5,4,4,2,0,2,3,2,5,2,2,1,0,4


In [8]:
# Extract the target dataframe
data_targets = data.drop([k for k in data_features], axis=1)  # dataframe with non-missing values

print(data_targets.shape)
data_targets.head()

(1000000, 55)


,F_1_0,F_1_1,F_1_2,F_1_3,F_1_4,F_1_5,F_1_6,F_1_7,F_1_8,F_1_9,F_1_10,F_1_11,F_1_12,F_1_13,F_1_14,F_3_0,F_3_1,F_3_2,F_3_3,F_3_4,F_3_5,F_3_6,F_3_7,F_3_8,F_3_9,F_3_10,F_3_11,F_3_12,F_3_13,F_3_14,F_3_15,F_3_16,F_3_17,F_3_18,F_3_19,F_3_20,F_3_21,F_3_22,F_3_23,F_3_24,F_4_0,F_4_1,F_4_2,F_4_3,F_4_4,F_4_5,F_4_6,F_4_7,F_4_8,F_4_9,F_4_10,F_4_11,F_4_12,F_4_13,F_4_14
0,-0.354591,-0.464038,2.304115,0.734486,1.696395,0.136285,-0.518344,0.502640,-1.852504,-0.500665,-1.416075,1.201521,0.551902,-0.759827,NaN,-0.240522,0.061529,0.561090,-0.171943,-0.431996,0.473508,0.596924,0.819306,1.479061,1.264616,-1.116881,0.759443,-0.086915,-0.620685,0.057216,1.076380,-0.780608,-1.940907,-0.717021,0.599093,0.498347,0.118770,-0.228913,NaN,0.301610,5.547214,1.066871,-0.134313,-0.101040,-0.660871,3.744152,0.794438,0.265185,-0.561809,0.196480,0.373434,6.206995,3.809505,1.236486,1.182055
1,1.380940,-0.499626,-0.418548,1.911725,-0.826130,-1.715371,-0.577091,-1.041486,0.596067,-0.363425,-0.853630,0.674525,0.843058,-0.041438,0.259496,-0.446068,1.433358,0.040099,-1.994062,-0.602324,-0.611391,-1.151884,0.065485,0.352023,-0.843751,1.167272,0.921445,-0.839827,0.759015,-1.547387,0.720435,-0.944045,1.796462,-1.046357,-0.581515,0.704543,0.375222,0.705963,0.032771,NaN,-1.707374,-1.188114,-0.562419,-1.462988,1.290672,-2.895826,-0.738275,2.361818,-0.060753,0.727249,-0.271882,5.232157,-4.218259,-2.724883,-0.063775
2,0.256023,-1.059874,NaN,0.345678,1.513814,1.243864,-0.509648,-0.800481,-0.115945,0.595777,-0.073235,-1.381605,-0.108676,0.703693,-0.464042,-0.770592,0.483139,-0.636484,-1.305018,-2.089889,0.276761,-1.208640,-0.855769,0.232363,0.215841,-1.031405,0.582437,-0.314639,-0.497409,0.489356,0.915049,-0.513410,0.904206,-0.056089,0.212927,-0.574126,-1.517749,-0.888472,0.142264,1.000822,1.914908,3.877128,NaN,0.358635,0.443973,2.252834,0.472496,2.491386,0.353381,-0.260682,-0.000833,-0.116457,-2.131747,3.661499,-0.131576
3,-0.728420,-2.432399,-2.453602,-0.020509,0.333397,0.086049,-1.787601,0.667011,0.761564,-2.217847,-0.618973,0.742112,0.494157,0.744673,-0.769181,-0.691020,-1.003026,0.868989,0.464920,NaN,0.499838,0.203723,-0.451576,-1.233499,0.903493,1.144558,-1.479893,-0.414316,0.024820,-0.976814,0.176633,-0.940022,-1.918049,1.506448,0.604388,0.578250,-0.122676,-0.711811,0.228523,0.999599,-2.638262,0.546676,0.865400,-0.857077,2.667105,2.004600,-4.664806,-0.847211,-0.264249,0.664334,-0.557868,8.499483,-4.738799,-3.054611,0.494152
4,0.590212,-0.066127,0.468009,-1.096038,0.119399,-1.809710,0.466358,-0.053196,-0.580320,-1.143500,1.338692,1.193330,1.038010,-0.763692,0.513466,1.196402,1.076537,0.486482,0.424305,0.475876,0.426978,-1.668823,-0.843056,1.008980,-0.704152,-0.894834,1.491608,-0.149938,1.256330,2.427308,0.878677,0.594314,0.380175,0.666499,-0.664403,1.276316,NaN,-1.276574,-0.768874,1.852815,-0.230342,-0.459019,1.128705,-0.748683,NaN,0.976937,2.558883,3.377724,0.846891,0.696032,0.554121,-5.979714,-2.869631,3.733057,-0.722943


## 4. Extract top5 features

In [9]:
# # Target column
# tc = 'F_1_2'

### - Option A. Using correlation

In [10]:
# # Check the correlation
# def abs_tarcor(target_col):
#     new_data = pd.concat([data_features, data.loc[:, target_col]], axis=1)
    
#     return abs(new_data.corr())   

In [11]:
# # List of top5 features
# def abs_top5_list(target_col):
#     abs_cor = abs_tarcor(target_col)
#     subject_col = abs_cor.loc[:, target_col]
#     subject_col = subject_col.drop(index = target_col)     # drop the row with the same feature
#     subject_col = subject_col.sort_values(ascending=False)

#     return subject_col[:5].index.to_list()

In [37]:
# # Combine the new features df and the target df
# def make_new_df(target_col):
    
#     new_features = data.loc[:, abs_top5_list(target_col)]
#     data_sub = pd.concat([new_features, data.loc[:, target_col]], axis=1)
    
#     return data_sub

In [40]:
# def make_new_df(target_col):

#     # Check the correlation
#     new_data = pd.concat([data_features, data.loc[:, target_col]], axis=1)
#     abs_cor = abs(new_data.corr())   

#     # List of top5 features
#     subject_col = abs_cor.loc[:, target_col]
#     subject_col = subject_col.drop(index = target_col)     # drop the row with itself
#     subject_col = subject_col.sort_values(ascending=False)
#     top5 = subject_col[:5].index.to_list()

#     # Combine the new features df and the target df
#     top5_features = data.loc[:, top5]
#     data_sub = pd.concat([top5_features, data.loc[:, target_col]], axis=1)

#     return data_sub


In [41]:
# new_df = make_new_df(tc)
# new_df.head()

,F_2_13,F_2_5,F_2_1,F_2_8,F_2_20,F_1_2
0,5,1,2,2,1,2.304115
1,4,2,5,0,1,-0.418548
2,2,1,2,1,1,NaN
3,1,1,3,6,6,-2.453602
4,4,0,0,1,2,0.468009


### - Option B. Using PCA

## 5. Split into train and test

In [14]:
# def train_test(target_col):
#     test_df = new_df[new_df[target_col].isnull()]
#     test_index = test_df.index
    
#     train_df = new_df.drop(index = test_index)  

#     X_train = train_df.drop(target_col, axis=1)
#     y_train = train_df[target_col]

#     X_test = test_df.drop(target_col, axis=1) 

#     return X_train, y_train, X_test

# X_train, y_train, X_test = train_test(tc)

In [78]:
def test_train(target_col):

    # Check the correlation
    new_data = pd.concat([data_features, data.loc[:, target_col]], axis=1)
    abs_cor = abs(new_data.corr())   

    # Get the list of top5 features
    subject_col = abs_cor.loc[:, target_col]
    subject_col = subject_col.drop(index = target_col)     # drop the row with itself
    subject_col = subject_col.sort_values(ascending=False)
    top5 = subject_col[:5].index.to_list()

    # Combine the new features df and the target df
    top5_features = data.loc[:, top5]
    data_sub = pd.concat([top5_features, data.loc[:, target_col]], axis=1)

    # Split into train and test
    test_df = data_sub[data_sub[target_col].isnull()]
    test_index = test_df.index
    train_df = data_sub.drop(index = test_index)  

    X_train = train_df.drop(target_col, axis=1)
    y_train = train_df[target_col]
    X_test = test_df.drop(target_col, axis=1) 

    return X_train, y_train, X_test

In [15]:
# print(X_train.shape)
# X_train.head()

In [16]:
# print(X_test.shape)
# X_test.head()

In [17]:
# print(y_train.shape)
# y_train.head()

## 6. Apply ML models

In [18]:
# def apply_ML(model):
#     model.fit(X_train, y_train)

#     # Predict
#     y_pred = model.predict(X_test)

#     # Add the row number
#     y_pred = pd.concat([pd.Series(X_test.index), pd.Series(y_pred)], axis=1)
#     y_pred = y_pred.rename(columns={0: "row", 1:tc})
#     y_pred = y_pred.set_index("row")

#     return y_pred
        

### 1) Linear Regression

In [ ]:
# Get y_pred
pred_LinReg = apply_ML(LinearRegression())

In [ ]:
# Create csv file
submit_LinReg = pred_LinReg.to_csv('submit_LinReg.csv', index=False)    # score : 

### 2) Ridge Regression

### 3) SVR(kernel='linear')

### 4) SVR(kernel='rbf')

### 5) Ensemble Regressors

## 7. Run over all missing features

In [23]:
missing_features = data_targets.columns.to_list()

In [ ]:
# def apply_ML_combine(model):
#     for f in missing_features:
#         X_train, y_train, X_test = train_test(f)
#         apply_ML(model)

In [67]:
# def apply_ML_combine(target_col, model):

#     X_train, y_train, X_test = test_train(target_col)

#     # Fit
#     model.fit(X_train, y_train)

#     # Predict
#     y_pred = model.predict(X_test)

#     # Add the row number
#     y_pred = pd.concat([pd.Series(X_test.index), pd.Series(y_pred)], axis=1)
#     y_pred = y_pred.rename(columns={0: "row", 1:target_col})
#     y_pred = y_pred.set_index("row")

#     return y_pred

In [108]:
def apply_ML(model):
    
    # Combine predicted values into a dataframe
    for target_col in missing_features:

        X_train, y_train, X_test = test_train(target_col)

        # Fit
        model.fit(X_train, y_train)

        # Predict
        y_pred = model.predict(X_test)

        # Add the row number
        y_pred = pd.concat([pd.Series(X_test.index), pd.Series(y_pred)], axis=1)
        y_pred = y_pred.rename(columns={0: "row", 1:target_col})
        y_pred = y_pred.set_index("row")

        if target_col=="F_1_0":
            combined_df = pd.DataFrame(y_pred)

        else:
            combined_df = pd.concat([combined_df,y_pred], axis=1)

    # Formatting
    m_list = []
    for row in combined_df.index.tolist():
        for col in [c for c in combined_df]:

            rc = str(row)+'-'+col
            value = combined_df.loc[row, col]

            if pd.notnull(value):        # if the value is not null
                m_list.append((rc, value))

    y_pred = pd.DataFrame(m_list)
    y_pred = y_pred.rename(columns={0: "row-col", 1:"value"})

    return y_pred

In [109]:
k = apply_ML(LinearRegression())

print(type(k))
k

<class 'pandas.core.frame.DataFrame'>


,row-col,value
0,0-F_1_14,0.001008
1,0-F_3_23,0.002437
2,1-F_3_24,0.001913
3,2-F_1_2,-0.001110
4,2-F_4_2,-0.086684
...,...,...
999995,999993-F_4_2,-0.082244
999996,999994-F_3_10,0.002705
999997,999994-F_4_9,-0.077922
999998,999997-F_3_14,0.000404


In [88]:
# Return 'row-col',value
def formatting(df):

    m_list = []

    for row in df.index.tolist():
        for col in [c for c in df]:

            rc = str(row)+'-'+col
            value = df.loc[row, col]

            if pd.notnull(value):        # if the value is not null
                m_list.append((rc, value))
                
    return m_list 

In [89]:
formatting(k)

[('0-F_1_14', 0.001007751401907557),
 ('0-F_3_23', 0.0024367444740756796),
 ('1-F_3_24', 0.0019126108694524843),
 ('2-F_1_2', -0.0011101567871441838),
 ('2-F_4_2', -0.08668362373609137),
 ('3-F_3_4', -0.0005047122320018449),
 ('4-F_3_21', -0.057273979439378134),
 ('4-F_4_4', 0.31541197261538684),
 ('5-F_1_1', 0.007659426649159777),
 ('5-F_3_11', -0.00587018780861036),
 ('5-F_3_14', -0.00396591022399324),
 ('7-F_4_3', -0.1986295067493145),
 ('7-F_4_14', 0.04149999831798708),
 ('10-F_3_0', 0.0005241999613483789),
 ('10-F_4_12', 0.32921271251574025),
 ('11-F_1_14', -0.00011797608571921525),
 ('11-F_3_17', -0.0004182277497321906),
 ('11-F_3_18', 0.0016433511527695036),
 ('12-F_1_14', 0.0008223641687875741),
 ('12-F_4_14', 0.03663867642677792),
 ('13-F_1_11', 0.0020812816037059836),
 ('14-F_1_13', -0.06256429278656966),
 ('14-F_4_3', -0.19571338011783024),
 ('17-F_3_1', -0.0024088174708052383),
 ('18-F_1_6', -0.0008887352349106609),
 ('19-F_1_11', -0.0068229901317206024),
 ('19-F_3_7', 0.00